In [11]:
from docplex.cp.model import *
from docplex.cp.config import get_default

In [12]:
from config import setup
setup()

In [13]:
n = 6

F = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
H = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
L= {}
pref_h = [
    [1,3,6,2,4,5],
    [4,6,1,2,5,3],
    [1,4,5,3,6,2],
    [6,5,3,4,2,1],
    [2,3,1,4,5,6],
    [3,1,2,6,5,4],
]
pref_f = [
    [1,5,6,3,2,4],
    [2,4,6,1,3,5],
    [4,3,6,2,5,1],
    [1,3,5,4,2,6],
    [3,2,6,1,4,5],
    [5,1,3,6,4,2],
]
for i, hlist in enumerate(pref_h):
    L['h'+str(i+1)]=['f'+str(j) for j in hlist]
for i, flist in enumerate(pref_f):
    L['f'+str(i+1)]=['h'+str(j) for j in flist]


mdl = CpoModel(name="Mariages")


In [14]:
#Lpos[h,f] postition de f dans L(h) + position de h dans L(h)
LPos = {}
for h in H:
    for f in F:
        LPos[h,f]=L[h].index(f)
        LPos[f,h]=L[f].index(h)


In [15]:
#x[h,f] mariage entre h et f
x = {}
z = {}
for h in H:
    for f in F:
        x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)
        z[h,f]=mdl.binary_var(name="z_" + h + "_" + f)

#y[i] cout du marriage de i (somme des couts du mariage pour les 2 partis) 
D = {}
for i in H+F:
    D[i]=mdl.integer_var(0,n-1,name="D_"+i)





In [16]:
#Ajout des contraintes

#Contrainte de monogamie
for h in H:
    mdl.add(sum([x[h,f] for f in F])==1)
for f in F:
    mdl.add(sum([x[h,f] for h in H])==1)


#Valeur de y (Position du partenaire dans le classement):
for h in H:
    mdl.add(D[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))
for f in F:
    mdl.add(D[f]==sum([(x[h,f]*LPos[f,h]) for h in H]))

In [17]:
#Couple bloquant =
#si (h,f) est un couple bloquant si 
#   - le partenaire actuel de h et en plus haute postion de preference que f
#   - et le partenaire actuel de f et en plus haute postion de preference que h
for h in H:
    for f in F:
        mdl.add((D[h]>LPos[h,f]) & (D[f]>LPos[f,h]) == false())
#mdl.add(sum(z[h,f] for h in H for f in F)==0)

In [21]:
DH = mdl.integer_var(name="DH")
DF = mdl.integer_var(name="DF")

mdl.add(DH==max([D[h] for h in H]))
mdl.add(DF==max([D[f] for f in F]))
mdl.add(minimize(abs(DH-DF)))


In [18]:
solution = mdl.solve()
solution.print_solution()



 ! ----------------------------------------------------------------------------
 ! Satisfiability problem - 48 variables, 60 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 67.0 (before), 67.0 (after)
 !  . Memory usage      : 301.4 kB (before), 301.4 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed            Branch decision
 *                     43  0.00s                  0  = x_h5_f4
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 43
 ! Number of fails        : 19
 ! Total memory usage     : 850.1 kB (802.0 kB CP Optimizer + 48.1 kB Concert)
 ! Time spent in solve    : 0.00s (0.00s en

In [22]:
print("\n SOLUTION :")
for h in H:
    for f in F:
        if(solution[x[h,f]]==1):
            print("Mariage entre ",h," et ",f," (Couts = ",LPos[h,f],LPos[f,h],")")



 SOLUTION :
Mariage entre  h1  et  f1  (Couts =  0 0 )
Mariage entre  h2  et  f2  (Couts =  3 0 )
Mariage entre  h3  et  f4  (Couts =  1 1 )
Mariage entre  h4  et  f6  (Couts =  0 4 )
Mariage entre  h5  et  f5  (Couts =  4 5 )
Mariage entre  h6  et  f3  (Couts =  0 2 )
